多条件搜索
方法一：使用sorted 函数

In [1]:
tuple_list = [[4,1,5], [4,3,2], [6,2,6], [4,2,7]]

In [2]:
sorted(tuple_list,key=lambda x:(x[0],x[1]))

[[4, 1, 5], [4, 2, 7], [4, 3, 2], [6, 2, 6]]

方法二：使用条件加权

In [3]:
tuple_list = [(4,1,5), (4,3,2), (6,2,6), (4,2,7)]

In [4]:
def get_score(x):
    return x[0]+0.1*x[1]+0.01*x[2]

In [5]:
sorted(tuple_list,key=get_score)

[(4, 1, 5), (4, 2, 7), (4, 3, 2), (6, 2, 6)]

百度百科爬虫（老师说只要五步）

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import random

In [2]:
url = "https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81/408485?fr=aladdin"

In [3]:
user_agent = "Mozilla/5.0 (Macintosh; U; Intel Mac OS x 10_6_8; en-us) AppleWebKit/534.50 (KHTM)" #这一步是什么意思？
headers = {'User-Agent': user_agent}

In [4]:
def get_all_subway_links(url):
    page = requests.get(url,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser',from_encoding="utf-8")
    links =set()
    for link in soup.findAll("table")[0].findAll("a"):
        links.add("http://baike.baidu.com/item/"+link.get_text())
    return list(links)    

In [5]:
links = get_all_subway_links(url)

In [6]:
links

['http://baike.baidu.com/item/北京地铁大兴线',
 'http://baike.baidu.com/item/北京地铁15号线',
 'http://baike.baidu.com/item/北京地铁S1线',
 'http://baike.baidu.com/item/北京地铁13号线',
 'http://baike.baidu.com/item/北京地铁14号线',
 'http://baike.baidu.com/item/北京地铁6号线',
 'http://baike.baidu.com/item/北京地铁9号线',
 'http://baike.baidu.com/item/北京地铁八通线',
 'http://baike.baidu.com/item/北京地铁燕房线',
 'http://baike.baidu.com/item/北京地铁西郊线',
 'http://baike.baidu.com/item/北京地铁4号线',
 'http://baike.baidu.com/item/北京地铁5号线',
 'http://baike.baidu.com/item/北京地铁昌平线',
 'http://baike.baidu.com/item/北京地铁7号线',
 'http://baike.baidu.com/item/北京地铁8号线',
 'http://baike.baidu.com/item/复八线',
 'http://baike.baidu.com/item/北京地铁亦庄线',
 'http://baike.baidu.com/item/北京地铁机场线',
 'http://baike.baidu.com/item/北京地铁10号线',
 'http://baike.baidu.com/item/北京地铁房山线',
 'http://baike.baidu.com/item/北京地铁16号线',
 'http://baike.baidu.com/item/北京地铁1号线']

In [7]:
def get_stations(text):
    stations = []
    pattern = re.compile(".*站")
    trs = text.find_all("tr")
    for tr in trs:
        if tr.a and tr.a.string:
            if pattern.search(tr.a.string):
                stations.append(tr.a.string)
    return stations

In [8]:
def get_stations_from_table(text):
    pattern = re.compile(r".*站")
    stations = []
    trs = text.find_all("tr")
    for tr in trs:
        if tr.a and tr.a.string:
            if pattern.search(tr.a.string):
                stations.append(tr.a.string)
    return stations    

In [13]:
subway = {}
pattern = re.compile(r'[\u4e00-\u9fa5_A-Z0-9]+')
for link in links:
    print(link)
    line = ''.join(pattern.findall(link))
    pattern_1 = re.compile(r"车站列表")
    pattern_2 = re.compile(r"车站列表")
    pattern_3 = re.compile(r"[\u4e00-\u9fa5-9+]")
    page = requests.get(link,headers=headers)
    soup = BeautifulSoup(page.content,'html.parser',from_encoding='utf-8')
    for table in soup.find_all("table"):
        if table.caption:
            caption = ''.join(pattern_3.findall(str(table.caption)))
            if pattern_1.search(caption) or pattern_2.search(caption):
                stations = get_stations_from_table(table)
                subway[line] = stations
                print(stations)
                break
                
        elif not table.caption:
            stations = get_stations(table)
            if len(stations) <= 1:
                continue
            else:
                subway[line] = stations
                print(stations)
                break
    print("_"*30)  
    

http://baike.baidu.com/item/北京地铁大兴线
['天宫院站', '生物医药基地站', '义和庄站', '黄村火车站', '黄村西大街站', '清源路站', '枣园站', '高米店南站', '高米店北站', '西红门站', '新宫站', '公益西桥站', '角门西站', '马家堡站', '北京南站', '陶然亭站', '菜市口站', '宣武门站', '西单站', '灵境胡同站', '西四站', '平安里站', '新街口站', '西直门站', '动物园站', '国家图书馆站', '魏公村站', '人民大学站', '海淀黄庄站', '中关村站', '北京大学东门站', '圆明园站', '西苑站', '北宫门站', '安河桥北站']
______________________________
http://baike.baidu.com/item/北京地铁15号线
['清华东路西口站', '六道口站', '北沙滩站', '奥林匹克公园站', '安立路站', '大屯路东站', '关庄站', '望京西站', '望京站', '望京东站', '崔各庄站', '马泉营站', '孙河站', '国展站', '花梨坎站', '后沙峪站', '南法信站', '石门站', '顺义站', '俸伯站']
______________________________
http://baike.baidu.com/item/北京地铁S1线
['苹果园站', '金安桥站', '四道桥站', '桥户营站', '上岸站', '栗园庄站', '小园站', '石厂站']
______________________________
http://baike.baidu.com/item/北京地铁13号线
['车公庄站', '西直门站', '大钟寺站', '知春路站', '五道口站', '上地站', '西二旗站', '龙泽站', '回龙观站', '霍营站', '立水桥站', '北苑站', '望京西站', '芍药居站', '光熙门站', '柳芳站', '东直门站']
______________________________
http://baike.baidu.com/item/北京地铁14号线
['园博园站', '大瓦窑站', '郭庄子站', '大井站', '七里庄站', '西局站

搜索问题
1.bfs 是否最优解？
若是点与点之间的cost不为负数则为最优解，否则不是。插入老师的图？？


In [26]:
from collections import defaultdict

In [27]:
simple_connection = {
    '北京': ['太原', '沈阳'],
    '太原': ['北京', '西安', '郑州'],
    '兰州': ['西安'],
    '郑州': ['太原',"长沙","石家庄","福州"],
    '西安': ['兰州',"长沙","太原"],
    '长沙': ['福州',"南宁","郑州","西安"],
    '沈阳': ['北京'],
    '福州': ['长沙','郑州'],    
}
graph = defaultdict(list)
graph.update(simple_connection)

In [28]:
line2station = {1:["兰州","西安","长沙","郑州"],
    2:["兰州","西安","太原","郑州","福州"],
    3:["兰州","西安","长沙","南宁"],
    4:["长沙","福州"],
    5:["兰州","西安","太原","北京","沈阳"],
    6:["郑州","石家庄"]
}

In [30]:
station2line = {}
for key,values in line2station.items():
    for v in values:
        if v not in station2line:
            station2line[v]={key}
        else:
            station2line[v].add(key)

In [31]:
station2line

{'兰州': {1, 2, 3, 5},
 '西安': {1, 2, 3, 5},
 '长沙': {1, 3, 4},
 '郑州': {1, 2, 6},
 '太原': {2, 5},
 '福州': {2, 4},
 '南宁': {3},
 '北京': {5},
 '沈阳': {5},
 '石家庄': {6}}

lines= {2,3,4}   ?insec = {3,4}
1{2,3,4},2{3,4,5}，3{1,2}

In [32]:
def get_trains(path):
    trans = 0
    lines = set(station2line[path[0]])
    for i, station in enumerate(path[:-1]):
        next_station = path[i+1]
        intersec = lines & station2line[next_station]
        if len(intersec) != 0: #交集的长度不为0
            lines = intersec
            continue
        else:
            lines = station2line[station] & station2line[next_station]
        trans += 1
    return trans

In [34]:
get_trains(["兰州","西安","长沙","郑州","太原","西安","长沙","福州"])

3

In [35]:
def get_len(x):
    return len(x)

In [36]:
def search(start, destination, connection_grpah,sorted_function):
    pathes = [[start]]
    
    while pathes:
        
        path = pathes.pop(0)
        froninter = path[-1]
        
        successors = connection_grpah[froninter]
        
        for city in successors:
            if city in path: continue
                
            new_path = path + [city]
            pathes.append(new_path)
        pathes = sorted(pathes,key=sorted_function)
        
        if pathes and (destination in pathes[0]):
            return pathes[0]               
                

In [38]:
print(search("兰州","福州",graph,get_trains))

['兰州', '西安', '太原', '郑州', '福州']


感谢李晓明老师的耐心解答，跌跌撞撞的终于运行完毕，复现了第二课的代码！还是很开心20190722